In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA


In [2]:
#  reading directory of pdfs
loader= PyPDFDirectoryLoader("./us_census")
documents= loader.load()

text_splitter= RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
final_doc= text_splitter.split_documents(documents)

In [3]:
# can use sentencetransformers too
embeddings = HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-small-en",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

c:\Users\ammar\OneDrive\Desktop\Git\Langchain-GenAI\venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [ ]:
import numpy as np

np.array(embeddings.embed_query(final_doc[0].page_content))

In [5]:
## VectorStore Creation
vectorstore=FAISS.from_documents(final_doc[:120],embeddings)

In [6]:
## Query using Similarity Search
query="WHAT IS HEALTH INSURANCE COVERAGE?"
relevant_docments=vectorstore.similarity_search(query)

print(relevant_docments[0].page_content)

2 U.S. Census Bureau
WHAT IS HEALTH INSURANCE COVERAGE?
This brief presents state-level estimates of health insurance coverage 
using data from the American Community Survey (ACS). The  
U.S. Census Bureau conducts the ACS throughout the year; the 
survey asks respondents to report their coverage at the time of 
interview. The resulting measure of health insurance coverage, 
therefore, reflects an annual average of current comprehensive 
health insurance coverage status.* This uninsured rate measures a 
different concept than the measure based on the Current Population 
Survey Annual Social and Economic Supplement (CPS ASEC). 
For reporting purposes, the ACS broadly classifies health insurance 
coverage as private insurance or public insurance. The ACS defines 
private health insurance as a plan provided through an employer 
or a union, coverage purchased directly by an individual from an 
insurance company or through an exchange (such as healthcare.


In [7]:
retriever=vectorstore.as_retriever(search_type="similarity",search_kwargs={"k":3})

In [8]:
import os
from dotenv import load_dotenv
load_dotenv()

# cant do that using getenv, have to add api here exact, make api with read permission, 
# os.environ['HUGGINGFACEHUB_API_TOKEN']="hf_"


In [10]:
from langchain_huggingface import HuggingFaceEndpoint

hf = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-v0.1",
    max_length=128,
    temperature=0.5,
    huggingfacehub_api_token="hf_"

)

hf.invoke("What is the health insurance coverage?")

WARNING! max_length is not default parameter.
                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\ammar\.cache\huggingface\token
Login successful


'\n\n- Health insurance is a type of insurance coverage that pays for medical and surgical expenses incurred by the insured. Health insurance can reimburse the insured for expenses incurred from illness or injury, or pay the care provider directly. It is often included in employer benefit packages as a means of enticing quality employees.\n\nWhat is the health insurance?\n\n- Health insurance is a type of insurance coverage that pays for medical and surgical expenses incurred by the insured. Health insurance can reimburse the insured for expenses incurred from illness or injury, or pay the care provider directly. It is often included in employer benefit packages as a means of enticing quality employees.\n\nWhat is the health insurance?\n\n- Health insurance is a type of insurance coverage that pays for medical and surgical expenses incurred by the insured. Health insurance can reimburse the insured for expenses incurred from illness or injury, or pay the care provider directly. It is o

In [11]:
# #Hugging Face models can be run locally through the HuggingFacePipeline class.
# from langchain_huggingface.llms import HuggingFacePipeline

# hf = HuggingFacePipeline.from_model_id(
#     model_id="mistralai/Mistral-7B-v0.1",
#     task="text-generation",
#     pipeline_kwargs={"temperature": 0, "max_new_tokens": 300},
#     huggingfacehub_api_token="hf_OU"
    
# )

# llm = hf 
# llm.invoke("What is the health insurance coverage?")

In [12]:
prompt_template="""
Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

{context}
Question:{question}

Helpful Answers:
 """

In [13]:
prompt=PromptTemplate(template=prompt_template,input_variables=["context","question"])

In [14]:
retrievalQA=RetrievalQA.from_chain_type(
    llm=hf,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt":prompt}
)

In [15]:
query="""DIFFERENCES IN THE
UNINSURED RATE BY STATE
IN 2022"""

In [16]:
# Call the QA chain with our query.
result = retrievalQA.invoke({"query": query})
print(result['result'])

1. In 2022, the uninsured rate in the group of states that expanded Medicaid eligibility was 22.7 percent, while the uninsured rate in the group of nonexpansion states was 18.0 percent.
2. The uninsured rate in the group of states that expanded Medicaid eligibility was 22.7 percent, while the uninsured rate in the group of nonexpansion states was 18.0 percent.
3. In 2022, the uninsured rate in the group of states that expanded Medicaid eligibility was 22.7 percent, while the uninsured rate in the group of nonexpansion states was 18.0 percent.
4. The uninsured rate in the group of states that expanded Medicaid eligibility was 22.7 percent, while the uninsured rate in the group of nonexpansion states was 18.0 percent.
5. The uninsured rate in the group of states that expanded Medicaid eligibility was 22.7 percent, while the uninsured rate in the group of nonexpansion states was 18.0 percent.
6. The uninsured rate in the group of states that expanded Medicaid eligibility was 22.7 percent,